In [11]:
import pandas as pd
from pandas_gbq import read_gbq

def get_notes_new(pat_id, project_id="hmh-datalake-dev"):
    query = f"""
    DECLARE end_date DATE;
    DECLARE start_date DATE;
    DECLARE patient_id STRING;

    SET end_date = DATE('2024-11-22'); -- Replace with your desired end date
    SET start_date = DATE('2022-11-22'); -- Replace with your desired start date
    SET patient_id = '{pat_id}'; -- Dynamically passed patient ID

    WITH filtered_notes AS (
        SELECT 
            hi.NOTE_ID, 
            hi.PAT_ID, 
            hi.PAT_ENC_CSN_ID,
            hi.NOTE_TYPE_NOADD_C,  -- Added to ensure availability in downstream CTEs
            nei.CONTACT_DATE, 
            nei.CONTACT_SERIAL_NUM, 
            nei.ENT_INST_LOCAL_DTTM,
            nei.NOTE_TYPE_C,
            nei.PRE_UCN_NOTE_TYPE_C, 
            hnt.NOTE_TEXT
        FROM `hmh-datalake-dev.CLARITY.HNO_INFO` hi
        INNER JOIN `hmh-datalake-dev.CLARITY.NOTE_ENC_INFO` nei
            ON nei.NOTE_ID = hi.NOTE_ID
        INNER JOIN `hmh-datalake-dev.CLARITY.HNO_NOTE_TEXT` hnt
            ON nei.CONTACT_SERIAL_NUM = hnt.NOTE_CSN_ID
        WHERE 
            hi.PAT_ID = patient_id
            AND nei.CONTACT_DATE BETWEEN start_date AND end_date
            AND nei.NOTE_LENGTH > 0
            AND nei.MOST_RECENT_CNCT_YN = 'Y'
            AND nei.NOTE_STATUS_C IN ('2', '3')
            AND hnt.NOTE_TEXT IS NOT NULL
            AND hi.DELETED_CAT_C IS NULL
    ),
    notes_with_types AS (
        SELECT 
            fn.PAT_ID,
            fn.PAT_ENC_CSN_ID,
            fn.NOTE_ID,
            fn.CONTACT_DATE,
            fn.ENT_INST_LOCAL_DTTM AS contact_datetime,
            fn.CONTACT_SERIAL_NUM AS note_csn_id,
            fn.NOTE_TEXT,
            fn.NOTE_TYPE_C,
            fn.PRE_UCN_NOTE_TYPE_C,
            COALESCE(znti.NAME, znt1.NAME, znt2.NAME, znth.NAME) AS note_type
        FROM filtered_notes fn
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_TYPE` znth 
            ON fn.NOTE_TYPE_NOADD_C = znth.NOTE_TYPE_C  -- Uses NOTE_TYPE_NOADD_C from filtered_notes
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_TYPE` znt1 
            ON fn.NOTE_TYPE_C = znt1.NOTE_TYPE_C
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_TYPE_IP` znti 
            ON fn.PRE_UCN_NOTE_TYPE_C = znti.TYPE_IP_C
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_TYPE` znt2 
            ON fn.PRE_UCN_NOTE_TYPE_C = znt2.NOTE_TYPE_C
    ),
    aggregated_notes AS (
        SELECT 
            PAT_ID,
            PAT_ENC_CSN_ID,
            note_csn_id,
            CONTACT_DATE,
            contact_datetime,
            note_type,
            STRING_AGG(NOTE_TEXT, ' ' ORDER BY contact_datetime, note_csn_id) AS note_text
        FROM notes_with_types
        GROUP BY 
            PAT_ID, PAT_ENC_CSN_ID, note_csn_id, CONTACT_DATE, contact_datetime, note_type
    )
    SELECT 
        PAT_ID,
        PAT_ENC_CSN_ID,
        note_csn_id,
        CONTACT_DATE,
        contact_datetime,
        note_type,
        note_text
    FROM aggregated_notes
    ORDER BY contact_datetime DESC, note_csn_id DESC
    LIMIT 100;
    """
    
    df = read_gbq(query, project_id=project_id)
    
    return df

In [3]:
import pandas as pd
from pandas_gbq import read_gbq

def get_notes_old(pat_id, project_id="hmh-datalake-dev"):
    query = f"""
    DECLARE end_date DATE;
    DECLARE start_date DATE;
    DECLARE patient_id STRING;

    SET end_date = DATE('2024-11-22'); -- Replace with your desired end date
    SET start_date = DATE('2022-11-22'); -- Replace with your desired start date
    SET patient_id = '{pat_id}'; -- Dynamically passed patient ID

    WITH notes_base AS (
        SELECT 
            hi.pat_id,
            hi.pat_enc_csn_id,
            nei.CONTACT_SERIAL_NUM AS note_csn_id,
            nei.CONTACT_DATE AS contact_date,
            nei.ENT_INST_LOCAL_DTTM AS contact_datetime,
            COALESCE(znti.NAME, znt1.NAME, znt2.NAME, znth.NAME) AS note_type,
            STRING_AGG(hnt.note_text, ' ' ORDER BY hnt.LINE) AS note_text
        FROM `hmh-datalake-dev.CLARITY.HNO_INFO` hi
        LEFT JOIN `hmh-datalake-dev.CLARITY.NOTE_ENC_INFO` nei 
            ON nei.NOTE_ID = hi.NOTE_ID
            AND nei.CONTACT_DATE BETWEEN start_date AND end_date
            AND hi.pat_id = patient_id
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_TYPE` znth 
            ON hi.NOTE_TYPE_NOADD_C = znth.NOTE_TYPE_C
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_TYPE` znt1 
            ON nei.NOTE_TYPE_C = znt1.NOTE_TYPE_C
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_TYPE_IP` znti 
            ON hi.IP_NOTE_TYPE_C = znti.TYPE_IP_C
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_TYPE` znt2 
            ON nei.PRE_UCN_NOTE_TYPE_C = znt2.NOTE_TYPE_C
        LEFT JOIN `hmh-datalake-dev.CLARITY.ZC_NOTE_STATUS` zns 
            ON nei.NOTE_STATUS_C = zns.NOTE_STATUS_C
        LEFT JOIN `hmh-datalake-dev.CLARITY.HNO_NOTE_TEXT` hnt 
            ON nei.CONTACT_SERIAL_NUM = hnt.NOTE_CSN_ID
        WHERE hnt.note_text IS NOT NULL
          AND nei.NOTE_LENGTH > 0
          AND nei.MOST_RECENT_CNCT_YN = 'Y'
          AND nei.NOTE_STATUS_C IN ('2', '3')
          AND hi.DELETED_CAT_C IS NULL
        GROUP BY 
            hi.pat_id, hi.pat_enc_csn_id, nei.CONTACT_SERIAL_NUM, nei.CONTACT_DATE, 
            nei.ENT_INST_LOCAL_DTTM, znti.NAME, znt1.NAME, znt2.NAME, znth.NAME
        ORDER BY contact_datetime DESC, note_csn_id DESC
        LIMIT 100
    )
    SELECT 
        notes_base.pat_id,
        notes_base.pat_enc_csn_id,
        notes_base.note_csn_id,
        notes_base.contact_date,
        notes_base.contact_datetime,
        notes_base.note_type,
        notes_base.note_text
    FROM notes_base;
    """
    
    df = read_gbq(query, project_id=project_id)
    
    return df


In [18]:
import pandas as pd
import time
from tqdm import tqdm 

def compare_queries(patient_ids, project_id="hmh-datalake-dev"):
    results = []

    for pat_id in tqdm(patient_ids, desc="Processing patients"):
        start_time_new = time.time()
        try:
            df_new = get_notes_new(pat_id, project_id) 
            runtime_new = time.time() - start_time_new
            count_new = len(df_new) 
            char_length_new = df_new['note_text'].str.len().sum() if 'note_text' in df_new.columns else 0  # Total character length
        except Exception as e:
            runtime_new = None
            count_new = None
            char_length_new = None
            print(f"Error with get_notes_new for patient {pat_id}: {e}")

     
        start_time_old = time.time()
        try:
            df_old = get_notes_old(pat_id, project_id) 
            runtime_old = time.time() - start_time_old
            count_old = len(df_old)  
            char_length_old = df_old['note_text'].str.len().sum() if 'note_text' in df_old.columns else 0  # Total character length
        except Exception as e:
            runtime_old = None
            count_old = None
            char_length_old = None
            print(f"Error with get_notes_old for patient {pat_id}: {e}")

       
        if runtime_new is not None and runtime_old is not None and runtime_old > 0:
            percentage_change = ((runtime_old - runtime_new) / runtime_old) * 100
        else:
            percentage_change = None

        # Append results
        results.append({
            "Patient ID": pat_id,
            "New Query Runtime (s)": runtime_new,
            "Old Query Runtime (s)": runtime_old,
            "Count New": count_new,
            "Count Old": count_old,
            "Char Length New": char_length_new,
            "Char Length Old": char_length_old,
            "Percentage Change (%)": percentage_change
        })
        
    results_df = pd.DataFrame(results)
    return results_df

# usage
if __name__ == "__main__":
    patient_ids = [
    'Z5771831',
    'Z7931652',
    'Z3892096',
    'Z4005123',
    'Z7216117',
    'Z5219649',
    'Z3953335',
    'Z6311090',
    'Z5334817',
    'Z8163952',
    'Z4687066',
    'Z5669267',
    'Z7033611',
    'Z5902242',
    'Z4885577',
    'Z6245796'
]

    comparison_results = compare_queries(patient_ids)
    comparison_results


Processing patients:   0%|          | 0/16 [00:00<?, ?it/s]/home/hakim_moazam_hmhn_org/docker/query_test/querycheck/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(
/home/hakim_moazam_hmhn_org/docker/query_test/querycheck/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functionality.
  record_batch = self.to_arrow(
Processing patients:   6%|▋         | 1/16 [00:38<09:41, 38.75s/it]/home/hakim_moazam_hmhn_org/docker/query_test/querycheck/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2379: UserWarning: A progress bar was requested, but there was an error loading the tqdm library. Please install tqdm to use the progress bar functi

In [19]:
comparison_results

,Patient ID,New Query Runtime (s),Old Query Runtime (s),Count New,Count Old,Char Length New,Char Length Old,Percentage Change (%)
0,Z5771831,17.440931,21.309534,5,5,13393,13393,18.154329
1,Z7931652,17.677679,24.627421,15,15,25738,25738,28.219530
2,Z3892096,14.232941,18.950782,3,3,11457,11457,24.895231
3,Z4005123,14.252706,19.779383,3,3,7204,7204,27.941604
4,Z7216117,15.442478,19.311265,14,14,35973,35973,20.033833
5,Z5219649,13.359561,21.135995,28,28,19956,19956,36.792372
6,Z3953335,22.481084,20.221067,2,2,9931,9931,-11.176550
7,Z6311090,18.151127,18.039339,3,3,10189,10189,-0.619688
8,Z5334817,13.013382,22.737146,6,6,32769,32769,42.765980
9,Z8163952,15.857792,57.993463,1,1,5322,5322,72.655898
